In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import json

In [ ]:

# Load data
train_data = pd.read_csv('/kaggle/input/chipsal-task3/SubTask-C-train.csv')
test_data = pd.read_csv('/kaggle/input/chipsal-task3/SubTask-C-(indextweet)test.csv')

##Different BERT Models


Run only the bert you want and configure as per requirements.

In [ ]:
# Set the model and tokenizer for MuRIL Large
model_name = "google/muril-large-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


In [ ]:
# Set the model and tokenizer for Verta BERT
model_name = "rahular/varta-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [ ]:
# Set the model and tokenizer for XLM-RoBERTa Large
model_name = "xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [ ]:

# Tokenize the data
def tokenize(batch):
    return tokenizer(batch['tweet'], padding=True, truncation=True, max_length=128)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['tweet'].tolist(), train_data['label'].tolist(), test_size=0.2, random_state=42
)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_data['tweet'].tolist(), truncation=True, padding=True, max_length=128)

class DevanagariDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])



train_dataset = DevanagariDataset(train_encodings, train_labels)
val_dataset = DevanagariDataset(val_encodings, val_labels)
test_dataset = DevanagariDataset(test_encodings)


In [ ]:
# Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:

# Helper function to ensure tensors are contiguous
def make_model_contiguous(model):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()
    return model

In [ ]:
# Training Arguments with Early Stopping
training_args = TrainingArguments(
    output_dir='./results_muril_large',
    num_train_epochs=13,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
    save_strategy="epoch",
    warmup_steps=400,
    weight_decay=0.06,
    gradient_accumulation_steps=8,
    load_best_model_at_end=True,
    metric_for_best_model="f1",

    greater_is_better=True,
    fp16=False,  # Disable mixed precision

)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

In [ ]:
# Make model contiguous and save
model = make_model_contiguous(model)
trainer.save_model('./best_model_muril_large')

In [ ]:
# Predict on test data
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(-1)

In [ ]:
# Save predictions
with open("predictions_muril_large.json", "w") as f:
    for idx, label in zip(test_data['index'], pred_labels):
        json.dump({"index": int(idx), "prediction": int(label)}, f)
        f.write("\n")

print("Predictions saved to predictions.json")


Transformers
